In [1]:
# This is necessary to recognize the modules
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

import asyncio
from decimal import Decimal
from core.data_sources import CLOBDataSource
from core.features.candles.volatility import VolatilityConfig
from research_notebooks.xtreet_bb.utils import generate_config, dump_dict_to_yaml

clob = CLOBDataSource()

2024-09-06 16:06:34,880 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x713d0265bd00>


In [2]:
# Screener parameters
CONNECTOR_NAME = "binance_perpetual"
INTERVALS = ["1m"]
DAYS = 7
FETCH_CANDLES = False
BATCH_CANDLES_REQUEST = 15
SLEEP_REQUEST = 60.0
VOLUME_THRESHOLD = 0.6  # From percentile VOLUME_THRESHOLD to 1
VOLATILITY_THRESHOLD = 0.6  # From percentile VOLATILITY_THRESHOLD to 1

# Trading Rules Filter
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 5  # In USDT
MAX_PRICE_STEP = 0.001  # Min price step in % (tick size)

VOLATILITY_WINDOW = 60  # In bars
VOLUME_FAST_WINDOW = 20  # No se usa
VOLUME_SLOW_WINDOW = 100  # No se usa

# Config generation
TOTAL_AMOUNT = 300  # General total amount for all markets
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_EXECUTORS_PER_SIDE = 1  # Maximum number of executors per side
COOLDOWN_TIME = 0
LEVERAGE = 20 # Should be for each trading pair
TIME_LIMIT = 60 * 60 * 24
BOLLINGER_LENGTHS = [50, 100, 200]
BOLLINGER_STDS = [1.4, 1.8, 2.2]
SL_STD_MULTIPLIER = 2
TS_DELTA_MULTIPLIER = 0.2
MAX_DCA_AMOUNT_RATIO = 2.8 # Amount 2 / Amount 1


# Config filtering
MIN_DISTANCE_BETWEEN_ORDERS = 0.01
MAX_TS_SL_RATIO = 0.5

# Download data
- Get trading rules
- Get candles for the last x days

In [3]:
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET)\
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE))\
    .get_all_trading_pairs()

if FETCH_CANDLES:
    number_of_calls = (len(trading_pairs) // BATCH_CANDLES_REQUEST) + 1

    all_candles = {}

    for i in range(number_of_calls):
        print(f"Batch {i + 1}/{number_of_calls}")
        start = i * BATCH_CANDLES_REQUEST
        end = (i + 1) * BATCH_CANDLES_REQUEST
        print(f"Start: {start}, End: {end}")
        end = min(end, len(trading_pairs))
        trading_pairs_batch = trading_pairs[start:end]

        tasks = [clob.get_candles_last_days(
        connector_name=CONNECTOR_NAME,
        trading_pair=trading_pair,
        interval=interval,
        days=DAYS,
        ) for trading_pair in trading_pairs_batch for interval in INTERVALS]

        candles = await asyncio.gather(*tasks)
        candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}
        all_candles.update(candles)
        if i != number_of_calls - 1:
            print(f"Sleeping for {SLEEP_REQUEST} seconds")
            await asyncio.sleep(SLEEP_REQUEST)
    clob.dump_candles_cache(os.path.join(root_path, "data"))
else:
    clob.load_candles_cache(os.path.join(root_path, "data"))
candles = [value for key, value in clob.candles_cache.items() if key[2] in INTERVALS and key[0] == CONNECTOR_NAME]

2024-09-06 16:06:36,767 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x713d01b00160>
2024-09-06 16:06:36,767 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x713d01cd2fe0>, 30392.064520291)]']
connector: <aiohttp.connector.TCPConnector object at 0x713d01b00190>


In [4]:
from core.features.candles.volume import VolumeConfig
from research_notebooks.xtreet_bb.utils import generate_screener_report

screner_report = generate_screener_report(
    candles=candles,
    trading_rules=trading_rules,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_FAST_WINDOW, long_window=VOLUME_FAST_WINDOW))
screner_report["url"] = screner_report["trading_pair"].apply(lambda x: f"https://www.binance.com/en/trade/{x}")
screner_report.sort_values("mean_natr", ascending=False, inplace=True)
screner_report

,trading_pair,mean_volatility,mean_natr,mean_bb_width,average_volume_per_hour,last_price,price_step_pct,normalized_score,url
110,NEIROETH-USDT,0.0097253,0.01611531,8.48136913,18042060.80929412,0.1396,0.00071633,1,https://www.binance.com/en/trade/NEIROETH-USDT
165,XEM-USDT,0.00436335,0.00636937,1.67299888,688599.73595992,0.0156,0.00641026,0.00758553,https://www.binance.com/en/trade/XEM-USDT
50,QUICK-USDT,0.00331915,0.00494724,3.89310795,7690179.50986999,0.04556,0.00021949,0.19516165,https://www.binance.com/en/trade/QUICK-USDT
24,REEF-USDT,0.00330923,0.00484884,3.26473049,14697234.52757105,0.00121,0.00082645,0.31305094,https://www.binance.com/en/trade/REEF-USDT
127,CRV-USDT,0.0031153,0.00463922,1.56157176,5049395.0506332,0.265,0.00377358,0.05174774,https://www.binance.com/en/trade/CRV-USDT
...,...,...,...,...,...,...,...,...,...
131,LOOM-USDT,0.00077972,0.00084157,0.83893234,139307.77892878,0.0412,0.00024272,0.00076103,https://www.binance.com/en/trade/LOOM-USDT
179,XLM-USDT,0.000554,0.00063495,0.60824518,750284.13649916,0.08781,0.00011388,0.00297217,https://www.binance.com/en/trade/XLM-USDT
160,TRX-USDT,0.00035611,0.00043862,0.39203352,3994274.07787786,0.1478,0.00006766,0.0102014,https://www.binance.com/en/trade/TRX-USDT
192,BTCDOM-USDT,0.00033708,0.00036044,0.33709196,132841.71419446,2857.1,0.000035,0.00029167,https://www.binance.com/en/trade/BTCDOM-USDT


In [14]:
# Calculate the 20th percentile (0.2 quantile) for both columns
natr_percentile = screner_report['mean_natr'].quantile(VOLATILITY_THRESHOLD)
volume_percentile = screner_report['average_volume_per_hour'].quantile(VOLUME_THRESHOLD)

# Filter the DataFrame to get observations where mean_natr is greater than its 20th percentile
# and average_volume_per_hour is greater than its 20th percentile
screener_top_markets = screner_report[
    (screner_report['mean_natr'] > natr_percentile) &
    (screner_report['average_volume_per_hour'] > volume_percentile) &
    (screner_report["price_step_pct"] < MAX_PRICE_STEP)
].sort_values(by="average_volume_per_hour").head(10)

# Display the filtered DataFrame
screener_top_markets[["trading_pair", "mean_natr", "average_volume_per_hour", "price_step_pct", "url"]]

,trading_pair,mean_natr,average_volume_per_hour,price_step_pct,url
101,BEAMX-USDT,0.00173747,727156.72267267,0.00008578,https://www.binance.com/en/trade/BEAMX-USDT
213,LISTA-USDT,0.00181937,746252.15146582,0.00031867,https://www.binance.com/en/trade/LISTA-USDT
200,ARKM-USDT,0.00183161,781407.58286378,0.00010626,https://www.binance.com/en/trade/ARKM-USDT
224,REZ-USDT,0.00172569,819931.22254819,0.00029931,https://www.binance.com/en/trade/REZ-USDT
70,BANANA-USDT,0.00174956,825320.76453259,0.00002651,https://www.binance.com/en/trade/BANANA-USDT
76,REN-USDT,0.00177216,828735.38704614,0.000293,https://www.binance.com/en/trade/REN-USDT
0,LEVER-USDT,0.00206584,840326.09548268,0.00005176,https://www.binance.com/en/trade/LEVER-USDT
189,VOXEL-USDT,0.00208666,883395.50190753,0.00071994,https://www.binance.com/en/trade/VOXEL-USDT
89,TURBO-USDT,0.00201473,1001940.45111763,0.00028506,https://www.binance.com/en/trade/TURBO-USDT
103,ZK-USDT,0.00175745,1006455.27349505,0.00010004,https://www.binance.com/en/trade/ZK-USDT


In [15]:
strategy_configs = generate_config(
    connector_name=CONNECTOR_NAME,
    intervals=INTERVALS,
    screener_top_markets=screener_top_markets,
    candles=candles,
    total_amount=TOTAL_AMOUNT,
    max_executors_per_side=MAX_EXECUTORS_PER_SIDE,
    cooldown_time=COOLDOWN_TIME,
    leverage=LEVERAGE,
    time_limit=TIME_LIMIT,
    bb_lengths=BOLLINGER_LENGTHS,
    bb_stds=BOLLINGER_STDS,
    sl_std_multiplier=SL_STD_MULTIPLIER,
    min_distance_between_orders=MIN_DISTANCE_BETWEEN_ORDERS,
    max_ts_sl_ratio=MAX_TS_SL_RATIO,
    ts_delta_multiplier=TS_DELTA_MULTIPLIER,
    max_dca_amount_ratio=MAX_DCA_AMOUNT_RATIO
)

Skipping ZK-USDT due to stop loss closer to get executed:BEP: 0.01725441007891848, SL: 0.011528565647386298, S1: 0.03184391179816236


In [16]:
len(set([config["trading_pair"] for config in strategy_configs]))

10

In [17]:
len(strategy_configs)

57

In [18]:
# DCA amounts
for config in strategy_configs:
    dump_dict_to_yaml("configs/", config)